In [1]:
import numpy as np

import igm

import h5py

from alabtools import HssFile

from igm.model import Model, Particle
from igm.restraints import Polymer, Envelope, Steric, intraHiC

(numexpr.utils) 2023-04-12 16:56:54,709 [INFO] Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
(numexpr.utils) 2023-04-12 16:56:54,710 [INFO] NumExpr defaulting to 8 threads.


In [2]:
igm.utils.simple_print()

Confirm that the package is installed correctly.


In [2]:
hssfilename = 'igm-model_mcrb_2.5MB.hss'

with HssFile(hssfilename, 'r') as hss:

    index = hss.index
    chrom = hss.get_index().chrom
    
    radii = hss.radii  # np.array(n_particles)
    crd = hss.coordinates  # np.array(n_particles, n_structures, 3)
    
n_particles = crd.shape[0]
print('Number of loci = ' + str(n_particles))

# explore the 'copy_index' object, which is crucial in all Assignment Steps
copy_index = index.copy_index
print('Number of haploid loci (number of I and J) = ' + str(len(copy_index)))

Number of loci = 2094
Number of haploid loci (number of I and J) = 1100


In [3]:
import pickle as pkl

# load the speckles data
speckles = pkl.load(open('speckles.pkl', 'rb'))

In [16]:
# gives the size of each chromosome in number of beads
# order: chr1_A, chr2_A, ..., chr19_A, chrX_A, chrY_A, chr1_B, chr2_B, ..., chr19_B
# (sexual chromosomes on copy A)
print(index.chrom_sizes)

# gives [ 0  0  0 ... 39 39 39]
chain_ids = np.concatenate( [ [i]*s for i, s in enumerate(index.chrom_sizes) ] )
print(chain_ids)


[79 73 65 63 61 60 59 52 50 53 49 49 49 50 42 40 38 37 25 69 37 79 73 65
 63 61 60 59 52 50 53 49 49 49 50 42 40 38 37 25]
[ 0  0  0 ... 39 39 39]
(2094, 100, 3)


In [25]:
# add the speckles as particles for each model of the HSS file

# In IGM, there isn't a data structure that contains all the models.
# This is because the models are passed to LAMMPS individually.
models = list()

# loop over all the models
for struct_id in range(hss.nstruct):
    
    # get the current hss model
    model = Model(uid=struct_id)
        
    # adds the real particles to the model from the HSS file
    for bead in range(hss.nbead):
        model.addParticle(crd[bead, struct_id], radii[bead],
                          Particle.NORMAL, chainID=chain_ids[bead])
    
    # adds the speckles as particles to the model
    for spe in speckles[struct_id]:
        spe_crd, spe_r = spe
        model.addParticle(spe_crd, spe_r, Particle.DUMMY_STATIC, chainID='speckle')
    
    models.append(model)

In [30]:
# model.particles is a list of Particle objects
for struct_id in range(hss.nstruct):
    assert len(models[struct_id].particles) == n_particles + len(speckles[struct_id])

In [34]:
# read in configuration file
config_file = './config.json'
cfg = igm.Config(config_file)

cfg["model"]["restraints"]["envelope"]["nucleus_radius"]

[3050.0, 2350.0, 2350.0]

In [ ]:
# Initialize the restraints list
all_restraints = []

# Include the excluded volume (steric) restraint
# define the excluded volume object
ex = Steric(cfg.get("model/restraints/excluded/evfactor"))

print(ex)   # this is a igm.restraint.restraint class/object

### CONFUSED ABOUT HOW _apply_model AND _apply METHODS (OF CLASS RESTRAINT) WORK
model.addRestraint(ex)      # add restraint to the model: which means, forces are actually added involving the particles
all_restraints.append(ex)    # keep track of restraints also

We realized that the steric restraint only works with NORMAL particles.

This means that we will have to code a new type of restraint, specific for speckles.

The idea would be to mark the speckles particles with chainID = 'speckles', and then in the restrain function filter out all the particles that don't have this chainID type.